# Prepare supplementary data with mutation summary info

This notebook generates Supplementary Datasets 1-3, plus the `auto_info` dataframe used by future notebooks.

In [1]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

import pandas as pd
import numpy as np
import math
import os
from collections import defaultdict
from statsmodels.stats.proportion import proportions_ztest
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.ticker import FormatStrFormatter
import matplotlib.pyplot as plt

import trtools.utils.utils

from mutation_pattern_utils import *

/Users/melissagymrek/opt/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#### Load mutation datasets ####
DATADIR = '../../BXD-STR-Mutator-Manuscript/outs'
auto = pd.read_csv(os.path.join(DATADIR, 'denovo_strs_filtered.csv'))

#### Load other metadata ###
motif_info = pd.read_csv(os.path.join(DATADIR, 'motif_info.csv'))
calls_info = pd.read_csv(os.path.join(DATADIR, 'all_repcn_proc_nosegdup_nolowcr_segreg.csv'))
chr13_gt = pd.read_csv(os.path.join(DATADIR, 'fou_gt_at_peak_chr13.tsv'), sep='\t')
strains_info = pd.read_csv(os.path.join(DATADIR, 'strain_info.csv'))
founder_labels = pd.read_csv(os.path.join(DATADIR, 'all_foulab_nosegdup_nolowcr_noalshared_padded_imp.csv'))

# Prepare locus-level summary of mutation info

In [3]:
#### Summarize missingness at each locus ####

# Note: missingness info needed to compute mutation rates, since we 
# need to know how many actual calls there were

# First, summarize missingness based on if the strain has sa B or D haplotype at the chr13 QTL
calls_info_missing = calls_info.iloc[:, 0:3]
calls_info_D = calls_info[list(chr13_gt[chr13_gt['fou_gt']=='D']['strain'])]
calls_info_missing.insert(3, 'missing_D', calls_info_D.isnull().sum(axis=1))
calls_info_missing['calls_D'] = calls_info_missing['missing_D'].apply(lambda x: len(calls_info_D.columns) - x )

calls_info_B = calls_info[list(chr13_gt[chr13_gt['fou_gt']=='B']['strain'])]
calls_info_missing.insert(5, 'missing_B', calls_info_B.isnull().sum(axis=1))
calls_info_missing['calls_B'] = calls_info_missing['missing_B'].apply(lambda x: len(calls_info_B.columns) - x )

# Now, summarize based on the founder haplotype on which the mutation falls
calls_info['chr_pos'] = calls_info.apply(lambda x: f'{x.chr}_{x.pos}', axis=1)
founder_labels['chr_pos'] = founder_labels.apply(lambda x: f'{x.chr}_{x.pos}', axis=1)
calls_info = calls_info[calls_info.chr_pos.isin(founder_labels.chr_pos)]
founder_labels = founder_labels[founder_labels.chr_pos.isin(calls_info.chr_pos)]
calls_info = calls_info.sort_values(['chr', 'pos'])
founder_labels = founder_labels.sort_values(['chr', 'pos'])

calls_BB = []
missing_BB = []
calls_DD = []
missing_DD = []
calls_BD = []
missing_BD = []
calls_DB = []
missing_DB = []

D_chr13 = list(chr13_gt[chr13_gt['fou_gt']=='D']['strain'])
B_chr13 = list(chr13_gt[chr13_gt['fou_gt']=='B']['strain'])

for index, row in calls_info.iterrows():
    B_col = founder_labels.columns[[x=='B' for x in founder_labels.iloc[index]]]
    D_col = founder_labels.columns[[x=='D' for x in founder_labels.iloc[index]]]
    BB_col = list(set(B_col) & set(B_chr13))
    DD_col = list(set(D_col) & set(D_chr13))
    BD_col = list(set(B_col) & set(D_chr13))
    DB_col = list(set(D_col) & set(B_chr13))

    calls_BB.append(row[BB_col].notnull().sum())
    missing_BB.append(row[BB_col].isnull().sum())
    calls_DD.append(row[DD_col].notnull().sum())
    missing_DD.append(row[DD_col].isnull().sum())
    calls_BD.append(row[BD_col].notnull().sum())
    missing_BD.append(row[BD_col].isnull().sum())
    calls_DB.append(row[DB_col].notnull().sum())
    missing_DB.append(row[DB_col].isnull().sum())
calls_info_missing.insert(7,'calls_BB', calls_BB)
calls_info_missing.insert(8,'missing_BB', missing_BB)
calls_info_missing.insert(9,'calls_DD', calls_DD)
calls_info_missing.insert(10,'missing_DD', missing_DD)
calls_info_missing.insert(11,'calls_BD', calls_BD)
calls_info_missing.insert(12,'missing_BD', missing_BD)
calls_info_missing.insert(13,'calls_DB', calls_DB)
calls_info_missing.insert(14,'missing_DB', missing_DB)

In [4]:
#### Attach motif info and other metadata to autosome dataframe ####

# Number of mutations per locus
num_mut = auto.groupby(["chr", "pos"]).size().to_frame('num_mut')
auto_info = pd.merge(auto, num_mut, on=['chr', 'pos'])

# Add motif information
auto_info = pd.merge(auto_info, motif_info[["chr","pos","end","motif","motif_len"]], on=["chr","pos","end"])
auto_info["motif"] = auto_info["motif"].apply(trtools.utils.utils.GetCanonicalMotif)

# Add founder at chr13 info for each strain
auto_info = pd.merge(auto_info, chr13_gt[['strain', 'fou_gt']], on=['strain'])
auto_info.rename(columns = {'fou_gt_y':'fou_gt_chr13'}, inplace = True)

# Summarize missingness
auto_info = pd.merge(auto_info, calls_info_missing, on=["chr","pos","end"])

# Add founder calls
calls_founder = calls_info[['chr', 'pos', 'end', 'DBA', 'C57BL']].copy()
calls_founder['DBA'] = calls_founder['DBA'].apply(GetFounderCall)
calls_founder['C57BL'] = calls_founder['C57BL'].apply(GetFounderCall)
auto_info = pd.merge(auto_info, calls_founder, on=['chr', 'pos', 'end'])
auto_info["fou_rn"] = auto_info["fou_gt_x"].apply(GetFounderCall)

# Add locus-level mutation summary stats
summary_ops = {
    "expan_perc": expan_perc,
    "num_B": num_B,
    "num_D": num_D,
    "num_B_founder": num_B_founder,
    "num_D_founder": num_D_founder,
    "num_DD": num_DD_founder_gt13,
    "num_DB": num_DB_founder_gt13,
    "num_BD": num_BD_founder_gt13,
    "num_BB": num_BB_founder_gt13,
    "num_expan": num_expan,
    "num_contr": num_contr
}
grouped_mut = auto_info.groupby(["chr", "pos"])
for key, val in summary_ops.items():
    df = grouped_mut.apply(val).to_frame(key)
    auto_info = pd.merge(auto_info, df, on=['chr','pos'])

In [5]:
#### Annotate expansion rate and size info by B/D ####
# expan_B, contr_B, expan_D, contr_D
expan_dict = defaultdict(lambda: [0,0,0,0])
# expan_BB, contr_BB, expan_DD, contr_DD, expan_BD, contr_BD, expan_DB, contr_DB
expan_gt13founder_dict = defaultdict(lambda: [0,0,0,0, 0,0,0,0])

expan_sizes_B = defaultdict(list)
expan_sizes_D = defaultdict(list)

for index, row in auto_info.iterrows():
    pos = f"{row['chr']}_{row['pos']}"
    if row['fou_gt_chr13'] == 'B':
        expan_sizes_B[pos].append(row['delta_fou']*row['expand_sign'])
        if row['expand_type'] == 'expan':
            expan_dict[pos][0] += 1
            #BB
            if row['founder'] =='B':
                expan_gt13founder_dict[pos][0] += 1
            #DB
            else: 
                expan_gt13founder_dict[pos][6] += 1
        #contractions
        else:
            expan_dict[pos][1] += 1
            if row['founder'] =='B':
                expan_gt13founder_dict[pos][1] += 1
            else: 
                expan_gt13founder_dict[pos][7] += 1
    else:
        expan_sizes_D[pos].append(row['delta_fou']*row['expand_sign'])
        if row['expand_type'] == 'expan':
            expan_dict[pos][2] += 1
            #DD
            if row['founder'] =='D':
                expan_gt13founder_dict[pos][2] += 1
            #BD
            else: 
                expan_gt13founder_dict[pos][4] += 1
        else:
            expan_dict[pos][3] += 1
            if row['founder'] =='D':
                expan_gt13founder_dict[pos][3] += 1
            else: 
                expan_gt13founder_dict[pos][5] += 1

auto_info['expan_B'] = 0
auto_info['contr_B'] = 0
auto_info['expan_D'] = 0
auto_info['contr_D'] = 0
auto_info['expan_sizes_B'] = np.empty((len(auto_info), 0)).tolist()
auto_info['expan_sizes_D'] = np.empty((len(auto_info), 0)).tolist()

auto_info['expan_BB'] = 0
auto_info['contr_BB'] = 0
auto_info['expan_DD'] = 0
auto_info['contr_DD'] = 0
auto_info['expan_BD'] = 0
auto_info['contr_BD'] = 0
auto_info['expan_DB'] = 0
auto_info['contr_DB'] = 0

for index, row in auto_info.iterrows():
    pos = f"{row['chr']}_{row['pos']}"
    auto_info.at[index, 'expan_B'] = expan_dict[pos][0]
    auto_info.at[index, 'contr_B'] = expan_dict[pos][1]
    auto_info.at[index, 'expan_D'] = expan_dict[pos][2]
    auto_info.at[index, 'contr_D'] = expan_dict[pos][3]
    auto_info.at[index, 'expan_sizes_B'] = expan_sizes_B[pos]
    auto_info.at[index, 'expan_sizes_D'] = expan_sizes_D[pos]
    
    auto_info.at[index, 'expan_BB'] = expan_gt13founder_dict[pos][0]
    auto_info.at[index, 'contr_BB'] = expan_gt13founder_dict[pos][1]
    auto_info.at[index, 'expan_DD'] = expan_gt13founder_dict[pos][2]
    auto_info.at[index, 'contr_DD'] = expan_gt13founder_dict[pos][3]

    auto_info.at[index, 'expan_BD'] = expan_gt13founder_dict[pos][4]
    auto_info.at[index, 'contr_BD'] = expan_gt13founder_dict[pos][5]
    auto_info.at[index, 'expan_DB'] = expan_gt13founder_dict[pos][6]
    auto_info.at[index, 'contr_DB'] = expan_gt13founder_dict[pos][7]

In [6]:
# Save auto_info for use in future notebooks
auto_info.to_csv("../outs/auto_info.csv")

# Output supplementary datasets 1-3

In [7]:
#### Supp dataset 1: All mutations ####
shared_cols = ["chr","pos","end", "motif", "motif_len", "DBA", "C57BL"]
supp_data1 = auto_info[shared_cols+["strain","RN_A","RN_B","founder", \
                                   "fou_gt_chr13","fou_rn", \
                                   "delta_fou","expand_sign","expand_type"]]

print("### Supplementary Dataset 1 ####")
print("Number of mutations: %s"%supp_data1.shape[0])
print("Number of unique strains: %s"%len(set(supp_data1["strain"])))
print("Number of unique loci: %s"%supp_data1[["chr","pos"]].drop_duplicates().shape[0])

# Output to a file
supp_data1.sort_values(["chr","pos"]).to_csv("../pdfs/SupplementaryDataset1.csv", index=False)

### Supplementary Dataset 1 ####
Number of mutations: 52812
Number of unique strains: 151
Number of unique loci: 18119


In [8]:
#### Supp dataset 2: Locus-level summary ####
auto_info["mut_sizes_B"] = auto_info["expan_sizes_B"].apply(lambda x: ",".join([str(item) for item in x]))
auto_info["mut_sizes_D"] = auto_info["expan_sizes_D"].apply(lambda x: ",".join([str(item) for item in x]))
supp_data2 = auto_info[shared_cols + \
                       ['num_mut', 'num_B', 'num_D', 'num_B_founder', 'num_D_founder', \
                        'num_DD', 'num_DB', 'num_BD', 'num_BB', \
                        'num_expan', 'num_contr', 'expan_B', 'contr_B', 'expan_D', 'contr_D', \
                        'mut_sizes_B', 'mut_sizes_D', \
                        'expan_BB', 'expan_DD', 'expan_DB', 'expan_BD', \
                        'contr_BB', 'contr_DD', 'contr_BD', 'contr_DB', \
                        'missing_D', 'calls_D', 'missing_B', 'calls_B', \
                        'calls_BB', 'calls_DD', 'calls_BD', 'calls_DB', \
                        'missing_BB', 'missing_DD', 'missing_BD', 'missing_DB']].drop_duplicates()

print("### Supplementary Dataset 2 ####")
print("Number of loci: %s"%supp_data2.shape[0])

# Output to a file
supp_data2.sort_values(["chr","pos"]).to_csv("../pdfs/SupplementaryDataset2.csv", index=False)

### Supplementary Dataset 2 ####
Number of loci: 18119


In [9]:
#### Supp dataset 3: Summary of call-missingness at all STRs analyzed ####
supp_data3 = calls_info_missing

print("### Supplementary Dataset 3 ####")
print("Number of loci: %s"%supp_data3.shape[0])

# Output to a file
supp_data3.sort_values(["chr","pos"]).to_csv("../pdfs/SupplementaryDataset3.csv", index=False)

### Supplementary Dataset 3 ####
Number of loci: 76634
